# Understanding ResNet architecture

Tout en construisant un réseau trop profond, il y a deux problèmes. En propagation vers l'avant, les dernières couches du réseau n'ont presque aucune information sur l'image d'origine. En rétropropagation, les premières couches proches de l'entrée n'obtiennent pratiquement aucune mise à jour des gradients en raison de disparition des gradients (en d'autres termes, elles sont presque nulles). Pour résoudre les deux problèmes, les réseaux résiduels (ResNet) utilisent une connexion de type autoroute qui transfère les informations brutes du quelques couches précédentes aux couches ultérieures. En théorie, même la dernière couche contiendra toutes les informations de l'image d'origine grâce à ce réseau autoroutier. Et en raison du saut de couches, les dégradés en arrière s'écouleront librement vers les couches initiales avec peu de modifications.

Le terme résiduel dans le réseau résiduel est l'information supplémentaire que le modèle doit apprendre de la couche précédente et qui doit être transmise à la couche suivante.

Un bloc résiduel typique apparaît comme suit :

<img src='https://ichi.pro/assets/images/max/724/1*VYudY7nDehi_26UZBqXJMQ.png' width=500px>

Comme vous pouvez le voir, alors que jusqu'à présent, nous nous sommes intéressés à extraire la valeur F(x), où x est la valeur provenant de la couche précédente, dans le cas d'un réseau résiduel, nous extrayons non seulement la valeur après avoir traversé les couches de poids, qui est F(x), mais additionnent également F(x) avec la valeur d'origine, qui est x.

Jusqu'à présent, nous avons utilisé des couches standard qui ont effectué des transformations linéaires ou de convolution F(x) ainsi qu'une activation non linéaire. Ces deux opérations détruisent dans un certain sens les informations d'entrée. Pour la première fois, nous voyons une couche qui non seulement transforme l'entrée, mais la préserve également, en ajoutant l'entrée directement à la transformation F(x) + x. De cette façon, dans certains scénarios, la couche a très peu de difficultés à se souvenir de l'entrée et peut se concentrer sur l'apprentissage de la transformation correcte pour la tâche.


Examinons plus en détail la couche résiduelle à travers le code en construisant un bloc résiduel

* **1.** Définir une classe avec l'opération de convolution (couche de poids dans le schéma précédent) dans la méthode __init__

In [4]:
from torch import nn
class ResLayer(nn.Module):
  def __init__(self,ni, no, kernel_size,stride=1):
    super(ResLayer, self).__init__()
    padding = kernel_size - 2
    self.conv = nn.Sequential(nn.Conv2d(ni, no, kernel_size, stride, padding=padding),
                              nn.ReLU())
                
  def forward(self, x):         
    x = self.conv(x) + x         
    return x
        

Notez que, dans le code précédent, nous avons défini le padding comme la dimension de la sortie lors du passage par convolution, et la dimension de l'entrée devrait rester la même si nous devions additionner les deux.

Dans le code précédent, nous obtenons une sortie qui est la somme de l'entrée passée par les opérations de convolution et de l'entrée d'origine.


Maintenant que nous avons appris comment fonctionnent les blocs résiduels, comprenons comment les blocs résiduels sont connectés dans un réseau pré-entraîné basé sur les blocs résiduels, ResNet18

<img src='https://www.researchgate.net/publication/342828449/figure/fig2/AS:911604103278592@1594354763036/ResNet-18-model-architecture-10.png' width=600px>

Comme vous pouvez le voir, il y a 18 couches dans l'architecture, c'est pourquoi elle est appelée architecture ResNet18. De plus, notez comment les connexions de saut sont établies à travers le réseau. Il n'est pas fait à chaque couche de convolution, mais après toutes les deux couches à la place




Comme nous l'avons fait dans VGG16, nous allons geler tous les différents modules, mais mettre à jour les paramètres dans les modules avgpool et fc à l'étape suivante

In [1]:
def get_model():
    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.fc = nn.Sequential(nn.Flatten(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 1),
    nn.Sigmoid())
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= 1e-3)
    return model.to(device), loss_fn, optimizer